<a href="https://colab.research.google.com/github/Ilektra-Giassa/Tutorials/blob/main/realEstate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How much more (inflation)?